In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install turicreate

In [ ]:
import turicreate as tc
sf = tc.SFrame.read_csv('/kaggle/input/jane-street-market-prediction/train.csv',
                        column_type_hints=float, header=True, delimiter=',')

In [ ]:
sf = sf.dropna(how='any')
sf = sf.filter_by(filter(lambda x : x > 0, sf['weight']), 'weight')

In [ ]:
sf['label'] = sf.select_columns(['weight', 'resp']).apply(lambda x: 1 if x['weight']*x['resp'] > 0 else 0)

In [ ]:
sf = sf.sample(0.1)

In [ ]:
sf_train, sf_test = sf.random_split(.7, seed=5)

In [ ]:
features = [x for x in sf.column_names()  if x.startswith('feature') ]

In [ ]:
import turicreate.aggregate as agg
date_count = sf.groupby(key_column_names='date', operations={'count': agg.COUNT()})
print(date_count.sort('count'))

In [ ]:
model = tc.regression.create(sf_train, target='resp',features=features)

In [ ]:
predictions = model.predict(sf_test)

In [ ]:
results = model.evaluate(sf_test)

In [ ]:
print(results)

In [ ]:
model.save('newturiclf')

In [ ]:
import janestreet
from turicreate import SFrame

env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

for (test_df, sample_prediction_df) in iter_test:
    X_test = SFrame(data=test_df.loc[:, test_df.columns.str.contains('feature')] )
    y_preds = model.predict(X_test)
    sample_prediction_df.action = list(map(lambda x: 1 if x>=0 else 0 , test_df['weight'] * y_preds))
    env.predict(sample_prediction_df)
    print("predicting")
    print(sample_prediction_df)